In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
%cd nlpProject/

/content/drive/MyDrive/nlpProject


In [4]:
%cd filter_programs

/content/drive/MyDrive/nlpProject/filter_programs


In [5]:
import nltk 
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/drive/MyDrive/nlpProject/all_reviews.csv')
shuffled_df = df.sample(frac =1)
shuffled_df.to_csv("shuffled_data.csv", index=False)

In [11]:

from nltk.corpus import stopwords
import glob
import json
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/nlpProject/all_reviews.csv')
shuffled_df = df.sample(frac =1)
shuffled_df.to_csv("shuffled_data.csv", index=False)

stops = stopwords.words('english')
punct = ["'", ".", ",", '"', ";", "?", "!", "-", "-", "(", ")", "*", "/", ":", "~"]

def get_data(name, answer):
  alltext = "" 
  toksents = []  
  stars = []
  all_files = glob.glob("../json_data/tokenize/first_10k/" + name + "/*")
  for file in all_files:
    f = open(file)
    data = json.loads(f.read().rstrip())
    f.close()
    allsent = []
    for review in data:
      sent = review["text"]
      allsent = sent_tokenize(sent)
      for w in allsent:
        if(w not in stops):
          toksents.append(nltk.word_tokenize(w))
          stars.append(review['stars'])
  if(answer == True):
    return(toksents)
  else:
    return(stars)
def get_all_data(answer):
  alltext = "" 
  toksents = []  
  stars = []
  f = open("../json_data/tokenize/first_10k/all_reviews.json")
  data = json.loads(f.read().rstrip())
  f.close()
  allsent = []
  for review in data:
    sent = review["text"]
    allsent = sent_tokenize(sent)
    for w in allsent:
      if(w not in stops):
        toksents.append(nltk.word_tokenize(w))
        stars.append(review['stars'])
  if(answer == True):
    return(toksents)
  else:
    return(stars)
def get_vectors(name):
  vectors = []
  data = get_data(name, True)
  model = Word2Vec(data, window=5, min_count=3, workers=4)
  for d in data:
    totvec = np.zeros(100)
    for w in d:
      if w.lower() in model:
        totvec = totvec + model[w.lower()]
    vectors.append(totvec)
  return vectors

In [12]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd 
from sklearn import metrics
from sklearn import tree
from sklearn.svm import LinearSVC

def knn():
  knn = KNeighborsClassifier()
  #call it something other than model
  knn.fit(get_vectors("train"), get_data("train", False))
  expected = get_data("test", False)
  predicted = knn.predict(get_vectors("test"))
  print(metrics.classification_report(expected, predicted))
  print(metrics.confusion_matrix(expected, predicted))
knn()

<ipython-input-11-197fc19ca984>:63: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w.lower() in model:
<ipython-input-11-197fc19ca984>:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  totvec = totvec + model[w.lower()]
<ipython-input-11-197fc19ca984>:63: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w.lower() in model:
<ipython-input-11-197fc19ca984>:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  totvec = totvec + model[w.lower()]


              precision    recall  f1-score   support

         1.0       0.22      0.48      0.30      3388
         2.0       0.20      0.07      0.10      3465
         3.0       0.23      0.18      0.20      3293
         4.0       0.19      0.13      0.16      3126
         5.0       0.18      0.17      0.18      2613

    accuracy                           0.21     15885
   macro avg       0.20      0.21      0.19     15885
weighted avg       0.20      0.21      0.19     15885

[[1629  215  551  472  521]
 [1661  229  551  471  553]
 [1508  261  604  422  498]
 [1419  243  560  415  489]
 [1169  202  415  371  456]]


In [14]:
def svc():
  svc = LinearSVC()
  svc.fit(get_vectors("train"), get_data("train", False))
  expected = get_data("test", False)
  predicted = svc.predict(get_vectors("test"))
  print(metrics.classification_report(expected, predicted))
  print(metrics.confusion_matrix(expected, predicted))
svc()

<ipython-input-11-197fc19ca984>:63: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w.lower() in model:
<ipython-input-11-197fc19ca984>:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  totvec = totvec + model[w.lower()]
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
<ipython-input-11-197fc19ca984>:63: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w.lower() in model:
<ipython-input-11-197fc19ca984>:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  totvec = totvec + model[w.lower()]


              precision    recall  f1-score   support

         1.0       0.21      0.93      0.34      3388
         2.0       0.20      0.04      0.07      3465
         3.0       0.25      0.00      0.00      3293
         4.0       0.00      0.00      0.00      3126
         5.0       0.00      0.00      0.00      2613

    accuracy                           0.21     15885
   macro avg       0.13      0.20      0.08     15885
weighted avg       0.14      0.21      0.09     15885

[[3167  214    0    0    7]
 [3310  151    2    0    2]
 [3157  135    1    0    0]
 [2998  127    1    0    0]
 [2503  110    0    0    0]]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
def dt():
  dt = tree.DecisionTreeClassifier()
  dt.fit(get_vectors("train"), get_data("train", False))
  expected = get_data("test", False)
  predicted = dt.predict(get_vectors("test"))
  print(metrics.classification_report(expected, predicted))
  print(metrics.confusion_matrix(expected, predicted))
dt()

<ipython-input-11-197fc19ca984>:63: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w.lower() in model:
<ipython-input-11-197fc19ca984>:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  totvec = totvec + model[w.lower()]
<ipython-input-11-197fc19ca984>:63: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if w.lower() in model:
<ipython-input-11-197fc19ca984>:64: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  totvec = totvec + model[w.lower()]


              precision    recall  f1-score   support

         1.0       0.25      0.04      0.07      3388
         2.0       0.21      0.04      0.07      3465
         3.0       0.17      0.09      0.12      3293
         4.0       0.20      0.71      0.31      3126
         5.0       0.20      0.15      0.17      2613

    accuracy                           0.20     15885
   macro avg       0.21      0.21      0.15     15885
weighted avg       0.21      0.20      0.14     15885

[[ 143  166  403 2268  408]
 [ 136  139  388 2381  421]
 [ 101  124  306 2390  372]
 [ 100  110  379 2204  333]
 [  94  113  364 1652  390]]
